<p style="text-align: center; font-size: 30px;">
    <strong>
          Scraper for Amazon.es
    </strong>
</p>

----

<b style="text-align: left; font-size: 18px;">
    The following project is specifically designed to scrape publicly available data concerning user reviews for goods listed on amazon.es
</b>

----

<b style="font-size: 20px;">Project Roadmap</b>

-  <p style="font-size: 18px;">Create a list of target URLs</p>
-  <p style="font-size: 18px;">Develop a program to automatically access specified websites and extract their data</p>
-  <p style="font-size: 18px;">Save the acquired data in a Database for future applications</p>
---

<b style="font-size: 20px;">Required tools</b>

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    <a href="https://www.python.org/downloads/" target="_blank" rel="noopener noreferrer">Latest version of Python</a>
</blockquote>

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    <a href="https://googlechromelabs.github.io/chrome-for-testing/" target="_blank" rel="noopener noreferrer">Latest version of WebDriver</a>
</blockquote>

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    Selenuim library for scraping purposes
</blockquote>

In [ ]:
!pip install selenium

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    Pandas library for data storage and manipulation
</blockquote>

In [ ]:
!pip install pandas

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    TQDM library for progress check 
</blockquote>

In [ ]:
!pip install tqdm

---
<b style="font-size: 20px;">Importing necessary libraries </b>


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os

import time
import datetime
import random
from tqdm import tqdm

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait


----
<b style="font-size: 20px;">
    Database outlook
</b>

<p style="font-size: 18px;">Our project requires data concerning coffee machines. They can by subdivided into 3 major groups:</p>
<ul>
    <li style="font-size: 18px;" class="list-item">Cafeteras de goteo</li>
    <li style="font-size: 18px;" class="list-item">Cafeteras automáticas</li>
    <li style="font-size: 18px;" class="list-item">Cafeteras individuales</li>
</ul>

<p style="font-size: 18px;">Each machine has its base page as well as a respective review section.</p>
<p style="font-size: 18px; margin-top: -20px;">Reviews for each machine will be stored separately, as show in the scheme below.</p>

---

![Diagram](assets/Scheme_light_final.png)

---
<b style="font-size: 20px;">
    Forming the base.csv
</b>

```python
dataframe = {
    'ID': ['0', '1', '2'],
    'category': ['Cafeteras de goteo', 'Cafeteras automaticas', 'Cafeteras individuales'],
    'category_url': [
        'https://www.amazon.es/s?i=kitchen&rh=n%3A2165180031&fs=true&page=1&qid=1721224661&ref=sr_pg_2',
        'https://www.amazon.es/s?i=kitchen&rh=n%3A2165187031&fs=true&page=1&qid=1721228721&ref=sr_pg_2',
        'https://www.amazon.es/s?i=kitchen&rh=n%3A2165185031&fs=true&page=1&qid=1721229794&ref=sr_pg_2'
    ]
}

df = pd.DataFrame(dataframe)
df.to_csv('database/base.csv', index=False)

df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
df.to_csv('database/0.csv')
df.to_csv('database/1.csv')
df.to_csv('database/2.csv')
```

----

<b style="font-size: 20px;">
    Database interaction 
</b>

<p style="font-size: 18px;">Defining functions for easy reading and writing of the data.</p>

In [4]:
def add_to_0(goods_url, reviews_url, filename=f'database/0.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
    
    next_id = os.path.join('', f'0-{len(df):06}')
    
    new_row = pd.DataFrame([{'ID': next_id, 'goods_url': goods_url, 'reviews_url': reviews_url}])
    
    if not df[(df['goods_url'] == goods_url) & (df['reviews_url'] == reviews_url)].empty:
        print(f"Duplicate row found. The row {goods_url} will not be added to {filename}.")
        return
    
    df = pd.concat([df, new_row], ignore_index=True)
    
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Row {goods_url} added successfully to {filename}")
    
def add_to_1(goods_url, reviews_url, filename=f'database/1.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
    
    next_id = os.path.join('', f'1-{len(df):06}')
    
    new_row = pd.DataFrame([{'ID': next_id, 'goods_url': goods_url, 'reviews_url': reviews_url}])
    
    if not df[(df['goods_url'] == goods_url) & (df['reviews_url'] == reviews_url)].empty:
        print(f"Duplicate row found. The row {goods_url} will not be added to {filename}")
        return
    
    df = pd.concat([df, new_row], ignore_index=True)
    
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Row {goods_url} added successfully to {filename}")
    
def add_to_2(goods_url, reviews_url, filename=f'database/2.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
    
    next_id = os.path.join('', f'2-{len(df):06}')
    
    new_row = pd.DataFrame([{'ID': next_id, 'goods_url': goods_url, 'reviews_url': reviews_url}])
    
    if not df[(df['goods_url'] == goods_url) & (df['reviews_url'] == reviews_url)].empty:
        print(f"Duplicate row found. The row {goods_url} will not be added to {filename}.")
        return
    
    df = pd.concat([df, new_row], ignore_index=True)
    
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Row {goods_url} added successfully to {filename}")

def get_reviews_url_by_index(index, filename): # getting reviews url string by ID

    df = pd.read_csv(filename)
    
    index = index.split('-')[1]
    index = int(index)
    
    if index < 0 or index >= len(df):
        raise IndexError("ID out of range of DataFrame")
    
    reviews_url = df.loc[index, 'reviews_url']
    
    return reviews_url

----

<b style="font-size: 20px;">
    Gathering and saving target URLs for each respective category
</b>

<p style="font-size: 18px;">In order to scrape the product reviews, we have to gather product URLs of interest.</p>
<p style="font-size: 18px;  margin-top: -20px;">Doing that automatically would bring in too much irrelevant goods from every category, like acessories or machines without reviews.</p>

In [27]:
add_to_1('https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/dp/B0CDCFH17J/ref=sr_1_5?dib=eyJ2IjoiMSJ9.hm_0gtZLV81iXXXtKmlJBukw1-YLVxl46pozcPTCJEZRyZspar_iwpBR3EVyK5U8HLvWZz3Qmtn8mB3LBO54S8ed-v54It4Uk4xz0w48XkLhIlGEKueoOlq4M-5PRtZuG4BUO8duJHKCxbHdmDp_GfYGniiZBw0DXFanlSBtrWiqW7oCEcTk8JvUrmRftutsXPTxOuuvYaDfE7la4mP84ffjke69eou__qOxUIFkWUbhw0xmxPv6zS837XSYanX71v1dlqenmNc8QK8WLuBsTxt32e0twlbyWWCYniU3uxo.vU_YhbI33x6j3-eUUiyH5BehdoVNq9NU-U5zwwUA3DM&dib_tag=se&qid=1723372286&s=kitchen&sr=1-5', 'https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/product-reviews/B0CDCFH17J/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

Row https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/dp/B0CDCFH17J/ref=sr_1_5?dib=eyJ2IjoiMSJ9.hm_0gtZLV81iXXXtKmlJBukw1-YLVxl46pozcPTCJEZRyZspar_iwpBR3EVyK5U8HLvWZz3Qmtn8mB3LBO54S8ed-v54It4Uk4xz0w48XkLhIlGEKueoOlq4M-5PRtZuG4BUO8duJHKCxbHdmDp_GfYGniiZBw0DXFanlSBtrWiqW7oCEcTk8JvUrmRftutsXPTxOuuvYaDfE7la4mP84ffjke69eou__qOxUIFkWUbhw0xmxPv6zS837XSYanX71v1dlqenmNc8QK8WLuBsTxt32e0twlbyWWCYniU3uxo.vU_YhbI33x6j3-eUUiyH5BehdoVNq9NU-U5zwwUA3DM&dib_tag=se&qid=1723372286&s=kitchen&sr=1-5 added successfully to database/1.csv


In [48]:
add_to_1('https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/dp/B0CDCFH17J/ref=pd_vtp_d_sccl_3_6/262-2739010-6039420?pd_rd_w=YSns8&content-id=amzn1.sym.79bfeeec-d048-49eb-a46d-fd0df43d59bb&pf_rd_p=79bfeeec-d048-49eb-a46d-fd0df43d59bb&pf_rd_r=EZBGN1Y8NJ7KGCWMN1SD&pd_rd_wg=CWYHE&pd_rd_r=f80789ca-030a-4a52-a3be-13ba5704a3a6&pd_rd_i=B0CDCFH17J&th=1', 'https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/product-reviews/B0CDCFH17J/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

Duplicate row found. The row https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/dp/B0CDCFH17J/ref=pd_vtp_d_sccl_3_6/262-2739010-6039420?pd_rd_w=YSns8&content-id=amzn1.sym.79bfeeec-d048-49eb-a46d-fd0df43d59bb&pf_rd_p=79bfeeec-d048-49eb-a46d-fd0df43d59bb&pf_rd_r=EZBGN1Y8NJ7KGCWMN1SD&pd_rd_wg=CWYHE&pd_rd_r=f80789ca-030a-4a52-a3be-13ba5704a3a6&pd_rd_i=B0CDCFH17J&th=1 will not be added to database/1.csv


----
<b style="font-size: 20px;">Specification of the target URLs</b>

<p style="font-size: 18px;"> Amazon.es limits the amount of publicly accessible product reviews to 100 per item.</p>
<p style="font-size: 18px; margin-top: -20px;">In order to possibly scrape more data one has to apply filters. That could potentially disclose more reviews.</p>

<p style="font-size: 18px;">The same method can be applied to other goods on amazon.es.</p>

In [5]:
def get_urls(ID, filename):
    
    try:
        base_url = get_reviews_url_by_index(ID, filename)
        print(f"The reviews_url at index {ID} is: {base_url}")
    except IndexError as e:
        print(e)
    
    filters = [
        'sortBy=recent',
        'sortBy=helpful',
        'sortBy=rating',
        'filterByStar=one_star',
        'filterByStar=two_star',
        'filterByStar=three_star',
        'filterByStar=four_star',
        'filterByStar=five_star'
    ]
    
    list_urls = []
    for filter_option in filters:
        for page in range(1, 11):
            
            #updating the filter and page number 
            updated_url = f"{base_url}&{filter_option}&pageNumber={page}"
            list_urls.append(updated_url)
    
    return list_urls

---
<b style="font-size: 20px;">Automatic interaction with web services</b>

<p style="font-size: 18px;">Selenium WebDriver is a tool that provides a programmatic interface for interacting with web browsers.</p>

```python
driver = webdriver.Chrome(service=service, options=options)
```

----
<b style="font-size: 20px;">Necessary adgustments for the bot</b>

<p style="font-size: 18px;">Before using WebDriver we have to add a few tweaks. </p>

In [6]:
options = webdriver.ChromeOptions() # making a ChromeOptions object

<p style="font-size: 18px;">To avoid sanctions the bot must emulate the behaviour of a human user.</p>

<p style="font-size: 18px; margin-top: -20px;">This setting allows the bot to send a user-agent string with browser specs, just like a human would.</p>

In [7]:
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

<p style="font-size: 18px;">Turn off Chrome UI and prepare to start the driver. Don't forget to enter the absolute path to your chromedriver.</p>
<p style="font-size: 18px; margin-top: -20px"><b>Don't forget to enter the absolute path to your chromedriver!</b></p>

In [8]:
options.add_argument('--headless')

DRIVER_PATH = '/Users/apple/Downloads/Project_Gnomi_Huekradi/assets/chromedriver' # Enter the absolute path to your chromedriver
service = Service(DRIVER_PATH)

----
<p style="text-align: center; font-size: 25px; margin-top: 20px;">
    <strong>
          The Final code for scraping
    </strong>
</p>

---
<p style="font-size: 18px;">Scraping reviews for last 10 items in Cafeteras individuales category as an example.</p>

In [47]:
def scrape_reviews(urls):

    driver = webdriver.Chrome(service=service, options=options)
    
    reviews = []
    
    for url in tqdm(urls, desc='Processing URLs', unit='URL'): 
        driver.get(url)

        time.sleep(1) # Waiting for the page to load

        # Getting reviews
        review_blocks = driver.find_elements(By.CSS_SELECTOR, '.a-section.review')
        for review_block in review_blocks:
            title = review_block.find_element(By.CSS_SELECTOR, '.review-title-content').text.strip()
            rating = review_block.find_element(By.CSS_SELECTOR, '.a-icon-alt').get_attribute('textContent').strip()
            body = review_block.find_element(By.CSS_SELECTOR, '[data-hook="review-body"]').text.strip()
            author = review_block.find_element(By.CSS_SELECTOR, '.a-profile-name').text.strip()
            date = review_block.find_element(By.CSS_SELECTOR, '.review-date').text.strip()

            reviews.append({
                'title': title,
                'rating': rating,
                'body': body,
                'author': author,
                'date': date,
            })
    
    driver.quit()
    return reviews

# Removing duplicates and saving data
def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.drop_duplicates(subset=['title', 'body', 'author'], inplace=True)
    df.to_csv(filename, index=False, encoding='utf-8')
    
    return int(df.shape[0])

if __name__ == "__main__":
    
    start_time = time.perf_counter() 

    for i in range(137, 144): #ID range
        
        ID = f'2-{i:06}'
    
        urls = get_urls(ID, f'database/2.csv')
        
        reviews = scrape_reviews(urls)
        print(f"Parsing item {i} is done")
    
        adress = f'database/reviews/2-{i:06}.csv'
        l = save_to_csv(reviews, adress)
        print(f'Saved {l} individual reviews in database/reviews/{ID}.csv\n')
    
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print('-'*50 + f'\nTime taken: {str(datetime.timedelta(seconds = elapsed_time))}')

The reviews_url at index 2-000137 is: https://www.amazon.es/Nescaf%C3%A9-Dolce-Gusto-KP1A3BKA-capacidad/product-reviews/B08HSJY2JT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|█████████████████████████| 80/80 [02:35<00:00,  1.94s/URL]


Parsing item 137 is done
Saved 370 individual reviews in database/reviews/2-000137.csv

The reviews_url at index 2-000138 is: https://www.amazon.es/Cafetera-c%C3%A1psulas-capacidad-calientes-Thermoblock/product-reviews/B0CK8B5PPZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|█████████████████████████| 80/80 [02:30<00:00,  1.88s/URL]


Parsing item 138 is done
Saved 500 individual reviews in database/reviews/2-000138.csv

The reviews_url at index 2-000139 is: https://www.amazon.es/Nespresso-DeLonghi-Inissia-EN-80-B/product-reviews/B00G5YOVZA/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


...

Parsing item 141 is done
Saved 371 individual reviews in database/reviews/2-000141.csv

The reviews_url at index 2-000142 is: https://www.amazon.es/Bosch-TAS1002N-Tassimo-Happy/product-reviews/B09CT39KZG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|█████████████████████████| 80/80 [02:41<00:00,  2.02s/URL]


Parsing item 142 is done
Saved 500 individual reviews in database/reviews/2-000142.csv

The reviews_url at index 2-000143 is: https://www.amazon.es/Bosch-TAS6502-Cafetera-c%C3%A1psulas-litros/product-reviews/B0857Z91PY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|█████████████████████████| 80/80 [02:18<00:00,  1.73s/URL]


Parsing item 143 is done
Saved 289 individual reviews in database/reviews/2-000143.csv

--------------------------------------------------
Time taken: 0:17:34.324899


---
<b style="font-size: 20px;">Demonstrating acquired data</b>


In [9]:
base_df = pd.read_csv('database/base.csv')
all_reviews = pd.DataFrame()
c = 0 # number of goods

for ID in base_df['ID']:
    id_df = pd.read_csv(f'database/{ID}.csv')
    category = base_df.loc[base_df['ID'] == ID, 'category'].values
    
    reviews_df = pd.DataFrame()
    
    for local_id in id_df['ID']:
        review_file = f'database/reviews/{local_id}.csv'
        c+=1
        
        if os.path.exists(review_file):
            review_df = pd.read_csv(review_file)
            reviews_df = pd.concat([reviews_df, review_df], ignore_index=True)
    
    print(f'Scraped {len(reviews_df)} reviews in {category[0]} category.')

    all_reviews = pd.concat([all_reviews, reviews_df], ignore_index=True)

print('-'*50 + f'\nTotal number of reviews in the database: {len(all_reviews)}\n' + '-'*50)

average_time = 139 #calculated based on previous attempts
print(f'Estimated total time taken: {str(datetime.timedelta(seconds = average_time * c))}')

all_reviews.head()

Scraped 72509 reviews in Cafeteras de goteo category.
Scraped 35330 reviews in Cafeteras automaticas category.
Scraped 42625 reviews in Cafeteras individuales category.
--------------------------------------------------
Total number of reviews in the database: 150464
--------------------------------------------------
Estimated total time taken: 20:16:15


,title,rating,body,author,date
0,Regalo,"5,0 de 5 estrellas",Me gustó mucho,Cliente Amazon,Revisado en España el 6 de abril de 2024
1,No ha durado ni la garantía + pésimo servicio ...,"1,0 de 5 estrellas","Ni los 24 meses de garantía ha durado, y al en...",MaX,Revisado en España el 3 de abril de 2024
2,Está bien,"3,0 de 5 estrellas",Después de más de dos meses usándola hace muy ...,Lorena,Revisado en España el 14 de marzo de 2024
3,Perfecta!,"5,0 de 5 estrellas",Me encanta! Soy adicta al café y me gusta toma...,Perfecta!,Revisado en España el 22 de febrero de 2024
4,EXCELENTE,"5,0 de 5 estrellas",como única pega que limpiar el palito del vapo...,Sil at,Revisado en España el 20 de febrero de 2024
